In [1]:
import re
import os
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from sklearn import metrics
import jieba
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from gensim.models import Word2Vec
from sklearn.svm import SVC

f:\python\python37\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# 中文字符显示
plt.rcParams['font.sans-serif']=['SimHei'] 
plt.rcParams['axes.unicode_minus']=False 

In [3]:
train = pd.read_csv('./cache/train.csv', index_col=0)
test = pd.read_csv('./cache/test.csv', index_col=0)

In [4]:
train.tokens = train.tokens.apply(eval)
test.tokens = test.tokens.apply(eval)

In [5]:
train.head(3)

,content,label,tokens
0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,体育,"[马晓旭, 意外, 受伤, 国奥, 警惕, 无奈, 大雨, 格外, 青睐, 殷家, 军, 记..."
1,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,体育,"[商瑞华, 首战, 复仇, 心切, 中国, 玫瑰, 美国, 方式, 攻克, 瑞典, 多曼来,..."
2,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,体育,"[冠军, 球队, 迎新, 欢乐, 派对, 黄旭获, 大奖, 张军, 赢, 下, PK, 赛,..."


In [6]:
# 词向量维度
VEC_SIZE = 128

w2v_model = Word2Vec(
    train.tokens.values, 
    vector_size=VEC_SIZE, 
    min_count=3, 
    seed=1234
)

In [7]:
def identity_tokenizer(text):
    return text


In [8]:
train.tokens.values[0]

['马晓旭',
 '意外',
 '受伤',
 '国奥',
 '警惕',
 '无奈',
 '大雨',
 '格外',
 '青睐',
 '殷家',
 '军',
 '记者',
 '傅亚雨',
 '沈阳',
 '报道',
 '来到',
 '沈阳',
 '国奥队',
 '依然',
 '没有',
 '摆脱',
 '雨水',
 '困扰',
 '7',
 '月',
 '31',
 '日',
 '下午',
 '6',
 '点',
 '国奥队',
 '日常',
 '训练',
 '再度',
 '受到',
 '大雨',
 '干扰',
 '无奈',
 '之下',
 '队员',
 '只',
 '慢跑',
 '25',
 '分钟',
 '草草收场',
 '31',
 '日',
 '上午',
 '10',
 '点',
 '国奥队',
 '奥体中心',
 '外场',
 '训练',
 '天',
 '阴沉沉',
 '气象预报',
 '显示',
 '当天',
 '下午',
 '沈阳',
 '大雨',
 '幸好',
 '队伍',
 '上午',
 '训练',
 '没有',
 '受到',
 '干扰',
 '下午',
 '6',
 '点',
 '球队',
 '抵达',
 '训练场',
 '时',
 '大雨',
 '已经',
 '下',
 '几个',
 '小时',
 '丝毫',
 '没有',
 '停下来',
 '意思',
 '抱',
 '试一试',
 '态度',
 '球队',
 '当天',
 '下午',
 '例行',
 '训练',
 '25',
 '分钟',
 '过去',
 '天气',
 '没有',
 '转好',
 '迹象',
 '保护',
 '球员',
 '国奥队',
 '决定',
 '中止',
 '当天',
 '训练',
 '全队',
 '立即',
 '返回',
 '酒店',
 '雨',
 '中',
 '训练',
 '足球队',
 '来说',
 '不是',
 '稀罕',
 '事',
 '奥运会',
 '即将',
 '之前',
 '全队',
 '变得',
 '娇贵',
 '沈阳',
 '最后',
 '一周',
 '训练',
 '国奥队',
 '保证',
 '现有',
 '球员',
 '不再',
 '出现意外',
 '伤病',
 '情况',
 '影响',
 '正式',
 '比赛',
 '这一',
 '阶段',
 '控

In [9]:
N = 1024
tfidf_model = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words=None, lowercase=False, max_features=N) 
tfidf_model.fit(train.tokens)
tfidf_id_to_tokens = {v: k for k, v in tfidf_model.vocabulary_.items()}

f:\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [10]:
tokens = train.tokens.values[0]
print(tokens)

['马晓旭', '意外', '受伤', '国奥', '警惕', '无奈', '大雨', '格外', '青睐', '殷家', '军', '记者', '傅亚雨', '沈阳', '报道', '来到', '沈阳', '国奥队', '依然', '没有', '摆脱', '雨水', '困扰', '7', '月', '31', '日', '下午', '6', '点', '国奥队', '日常', '训练', '再度', '受到', '大雨', '干扰', '无奈', '之下', '队员', '只', '慢跑', '25', '分钟', '草草收场', '31', '日', '上午', '10', '点', '国奥队', '奥体中心', '外场', '训练', '天', '阴沉沉', '气象预报', '显示', '当天', '下午', '沈阳', '大雨', '幸好', '队伍', '上午', '训练', '没有', '受到', '干扰', '下午', '6', '点', '球队', '抵达', '训练场', '时', '大雨', '已经', '下', '几个', '小时', '丝毫', '没有', '停下来', '意思', '抱', '试一试', '态度', '球队', '当天', '下午', '例行', '训练', '25', '分钟', '过去', '天气', '没有', '转好', '迹象', '保护', '球员', '国奥队', '决定', '中止', '当天', '训练', '全队', '立即', '返回', '酒店', '雨', '中', '训练', '足球队', '来说', '不是', '稀罕', '事', '奥运会', '即将', '之前', '全队', '变得', '娇贵', '沈阳', '最后', '一周', '训练', '国奥队', '保证', '现有', '球员', '不再', '出现意外', '伤病', '情况', '影响', '正式', '比赛', '这一', '阶段', '控制', '训练', '受伤', '控制', '感冒', '疾病', '出现', '队伍', '放在', '相当', '重要', '位置', '抵达', '沈阳', '之后', '中', '后卫', '冯萧霆', '一直', '没有', '训练', '冯萧霆', '7', '月', '

In [11]:
def build_weighted_feature(tokens):
    tfidf_feature = tfidf_model.transform([tokens])
    tfidf_feature = dict(tfidf_feature.todok().items())
    tfidf_feature = {k[1]: v for k, v in tfidf_feature.items()}
    
    feature = np.zeros(VEC_SIZE)
    d = 0
    # 对于每一个词
    for token_id, tfidf_weight in tfidf_feature.items():
        # 该词必须要有词向量，如果没有就跳过该词
        token = tfidf_id_to_tokens[token_id]
        if token not in w2v_model.wv:
            continue
        vector = w2v_model.wv[token]
        feature += tfidf_weight * vector
        d += tfidf_weight
    if d:
        feature /= d
    return feature

In [12]:
def build_baseline_feature(tokens):
    feature = np.zeros(VEC_SIZE)
    n = 0
    for token in tokens:
        if token in w2v_model.wv:
            feature += w2v_model.wv[token]
            n += 1
    if n:
        feature /= n
    return feature

In [13]:
train['baseline_feature'] = train.tokens.apply(build_baseline_feature)
test['baseline_feature'] = test.tokens.apply(build_baseline_feature)

train['weighted_feature'] = train.tokens.apply(build_weighted_feature)
test['weighted_feature'] = test.tokens.apply(build_weighted_feature)

In [14]:
train_features = tfidf_model.fit_transform(train.tokens).toarray()
test_features = tfidf_model.transform(test.tokens).toarray()

In [15]:
y_train = train.label.values
y_test = test.label.values

In [16]:
indices = defaultdict(list)
for index, label in enumerate(y_train):
    indices[label].append(index)
    
n = 10
selected_indices = []
for label, label_indices in indices.items():
    selected_indices.extend(np.random.choice(label_indices, n, replace=False))

In [17]:
len(selected_indices)

100

baseline word2vec特征，小样本结果

In [18]:
feature = 'baseline_feature'

In [35]:
X_train = np.vstack(train[feature].values)
X_test = np.vstack(test[feature].values)

In [37]:
model = LogisticRegression(max_iter=500)
model.fit(X_train[selected_indices], y_train[selected_indices])
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.91      0.98      0.94      1000
          娱乐       0.85      0.93      0.89      1000
          家居       0.76      0.30      0.44      1000
          房产       0.63      0.77      0.69      1000
          教育       0.96      0.70      0.81      1000
          时尚       0.80      0.91      0.85      1000
          时政       0.86      0.81      0.83      1000
          游戏       0.77      0.85      0.81      1000
          科技       0.73      0.72      0.72      1000
          财经       0.73      0.96      0.83      1000

    accuracy                           0.79     10000
   macro avg       0.80      0.79      0.78     10000
weighted avg       0.80      0.79      0.78     10000



baseline tfidf特征，小样本结果

In [38]:
len(tfidf_model.vocabulary_)

1024

In [39]:
model = LogisticRegression()
model.fit(train_features[selected_indices], y_train[selected_indices])
y_test_pred = model.predict(test_features)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.95      0.98      0.97      1000
          娱乐       0.92      0.88      0.90      1000
          家居       0.58      0.44      0.50      1000
          房产       0.70      0.80      0.74      1000
          教育       0.94      0.75      0.83      1000
          时尚       0.69      0.91      0.79      1000
          时政       0.81      0.81      0.81      1000
          游戏       0.90      0.85      0.88      1000
          科技       0.69      0.65      0.67      1000
          财经       0.88      0.98      0.93      1000

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.80     10000
weighted avg       0.81      0.80      0.80     10000



加权特征，小样本结果

In [40]:
feature = 'weighted_feature'
X_train = np.vstack(train[feature].values)
X_test = np.vstack(test[feature].values)

In [44]:
model = LogisticRegression(max_iter=100)
model.fit(X_train[selected_indices], y_train[selected_indices])
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

f:\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

          体育       0.91      0.98      0.94      1000
          娱乐       0.85      0.93      0.89      1000
          家居       0.76      0.30      0.44      1000
          房产       0.63      0.77      0.69      1000
          教育       0.96      0.70      0.81      1000
          时尚       0.80      0.91      0.85      1000
          时政       0.86      0.81      0.83      1000
          游戏       0.77      0.85      0.81      1000
          科技       0.73      0.72      0.72      1000
          财经       0.73      0.96      0.83      1000

    accuracy                           0.79     10000
   macro avg       0.80      0.79      0.78     10000
weighted avg       0.80      0.79      0.78     10000



https://www.heywhale.com/mw/dataset/5de4b6d0ca27f8002c4c530a

In [58]:
model = LogisticRegression()
model.fit(X_train[selected_indices], y_train[selected_indices])
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

f:\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

          体育       0.81      0.94      0.87      1000
          娱乐       0.75      0.83      0.79      1000
          家居       0.53      0.13      0.20      1000
          房产       0.56      0.69      0.62      1000
          教育       0.80      0.66      0.72      1000
          时尚       0.58      0.74      0.65      1000
          时政       0.67      0.76      0.71      1000
          游戏       0.64      0.59      0.62      1000
          科技       0.16      0.09      0.12      1000
          财经       0.61      0.93      0.74      1000

    accuracy                           0.64     10000
   macro avg       0.61      0.64      0.60     10000
weighted avg       0.61      0.64      0.60     10000



In [197]:
test['feature'] = test.tokens.apply(to_feature)
X_test = np.vstack(test.feature.values)

In [221]:
n = 100
selected_indices = []
for label, label_indices in indices.items():
    selected_indices.extend(np.random.choice(label_indices, n, replace=False))

In [222]:
model = LogisticRegression()
model.fit(X_train[selected_indices], y_train[selected_indices])

f:\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [223]:
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.78      0.74      0.76      1000
          娱乐       0.71      0.64      0.67      1000
          家居       0.43      0.08      0.13      1000
          房产       0.42      0.48      0.45      1000
          教育       0.40      0.50      0.44      1000
          时尚       0.73      0.71      0.72      1000
          时政       0.54      0.56      0.55      1000
          游戏       0.45      0.54      0.49      1000
          科技       0.61      0.71      0.65      1000
          财经       0.44      0.53      0.48      1000

    accuracy                           0.55     10000
   macro avg       0.55      0.55      0.53     10000
weighted avg       0.55      0.55      0.53     10000



In [151]:
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.82      0.85      0.84      1000
          娱乐       0.77      0.79      0.78      1000
          家居       0.56      0.16      0.24      1000
          房产       0.53      0.60      0.56      1000
          教育       0.50      0.56      0.53      1000
          时尚       0.77      0.85      0.80      1000
          时政       0.64      0.67      0.65      1000
          游戏       0.58      0.60      0.59      1000
          科技       0.71      0.75      0.73      1000
          财经       0.62      0.73      0.67      1000

    accuracy                           0.66     10000
   macro avg       0.65      0.66      0.64     10000
weighted avg       0.65      0.66      0.64     10000

